# Installing Necessary Libs

In [ ]:
!pip install -q keras

In [ ]:
!pip install -q pydrive

# Google Drive Access

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Specifying Project Path

In [ ]:
# Specifying the project path (where all the corresponding files exist)
project_path = '/content/drive/My Drive/Code Documentation Project/'

# Importing necessary libs

In [ ]:
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout
from keras.layers import LSTM,Conv1D, Bidirectional
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras import optimizers
from keras.layers import TimeDistributed
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder
from keras import callbacks
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score,jaccard_similarity_score, hamming_loss

from sklearn.preprocessing import LabelEncoder
import glob

import os

import gc
import keras.backend as K
import numpy as np

# Defining Some Necessary Functions

In [ ]:


def clean_text(text):
    import nltk
    nltk.download('stopwords')
    ## Remove puncuation
    #text = text.translate(string.punctuation)
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)



    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)

    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)


    return text

# Defining Model Architecture

In [ ]:
def create_model_BiLSTM(vocabulary_size=400000,embedding_size=100,embedding_matrix,num_class=5):
    ## create model
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix],trainable=False))

    model.add(Bidirectional(LSTM(300)))


    model.add(Dense(num_class, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()
    return model

# Loading Dataset

In [ ]:
# Loading old dataset
prev_df = pd.read_excel(project_path + 'Documentation Smell (Extension)/Dataset/labelled_dataset_full_SANER.xlsx')


# Loading new dataset
# reading the file names of new df
all_files = glob.glob(project_path+"Documentation Smell (Extension)/Dataset/all labelled sample sets-extension/" + "/*.xlsx")

new_df = pd.DataFrame()

# loading new file contents in the new_df
for filename in all_files:
    df = pd.read_excel(filename)
    try:
      df['Documentation Text'] = df['Documentation Text'].apply(clean_text)
    except:
      print(filename)
      pass
    new_df = new_df.append(df,ignore_index=True)

# clearing memory
del all_files
del df


# Training Set (Old dataset)

In [ ]:
texts_train=prev_df['Documentation Text']
texts_train=texts_train.map(lambda x: clean_text(x))

y_train=prev_df.iloc[:,1:6].values

vocabulary_size = 400000
#***********
time_step=300
embedding_size=100



tokenizer_train=Tokenizer(num_words=vocabulary_size)
tokenizer_train.fit_on_texts(texts_train)
encoded_train=tokenizer_train.texts_to_sequences(texts=texts_train)
#print(encoded_docs)
vocab_size_train = len(tokenizer_train.word_index) + 1
print(vocab_size_train)

X_train = sequence.pad_sequences(encoded_train, maxlen=time_step,padding='post')


# Glove Embedding

In [ ]:
f = open(project_path + 'glove.6B.100d.txt',encoding='utf-8')
embeddings_train={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_train[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_train))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size_train, embedding_size))
for word, i in tokenizer_train.word_index.items():
	embedding_vector_train = embeddings_train.get(word)
	if embedding_vector_train is not None:
		embedding_matrix[i] = embedding_vector_train

# Training Model

In [ ]:
vocabulary_size=embedding_matrix.shape[0]
model=create_model_BiLSTM(vocabulary_size,embedding_size,embedding_matrix, num_class=5)
history=model.fit(X_train,y_train,batch_size=256,epochs=10,validation_split=0.1,shuffle=True)

# Test set (new dataset)

In [ ]:
texts_test=new_df['Documentation Text']
texts_test=texts_test.map(lambda x: clean_text(x))

y_test=new_df.iloc[:,1:6].values


X_test = sequence.pad_sequences(encoded_train, maxlen=time_step,padding='post')

# Prediction with model

In [ ]:
pred=model.predict(X_test)


pred_binary=np.array(pred)
for i in range(len(pred_binary)):
  for j in range(len(pred_binary[i])):
    pred_binary[i][j]=int(1*(pred_binary[i][j]>0.5))

# Result Generation

In [ ]:
y_true_np = np.array(y_test)
y_pred_np = np.array(pred_binary)

num_classes = 5
for i in range(num_classes):
  print('Class: ' + str(i))
  new_y_true = y_true_np[:,i]
  new_y_pred = y_pred_np[:,i]

  print('Accuracy for class ' + str(i) + ': ' + str(accuracy_score(new_y_true,new_y_pred)))
  print('Classification Report for class' + str(i) + ': ' + str(classification_report(new_y_true,new_y_pred)))